## Summary:

The goal of this model is to try and fit the parity function. In other words, we will try to see how accurately can our model act like the Parity Function. The parity function can be defined as:

$
f : x \in \{0, 1\}^{\otimes n} \rightarrow y = 
    \begin{cases} 
      \text{1 if uneven number of 1's in x}\\
      \text{0 else} 
    \end{cases}
$

As an example, if $x$ is $1100$, the output of the parity function will be $f(1100) = 0$. If $x$ is $1110$, the output of the parity function will be $f(1110) = 1$. We want to train our model in such a way that it acts like the parity function.

From the model's perspective, the only thing model has the input and output labels for the bit-strings. It tries to map the input bit-strings to the corresponding output labels by finding the function that suits to the mapping. We do this on a subset of the available dataset. Once the training is done, we evaluate the model based on the other subset to see how accurate the formed mapping function is.

## Workflow:

### 1. Initialization

In [78]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device('default.qubit')

Here, we import the requied libraries: $pennylane$ for all the Pennylane functions and $numpy$ for comfortable processing of mathematical data

Then, we initialize the device to be a simple-state simulator of qubit-based quantum systems. Other options are $lightning.qubit$, for a more performant system, $default.mixed$ for a mixed-state simulator of qubit-based quantum systems.

### 2. Input

In [79]:
def state_preparation(x):
    qml.BasisState(x, wires=[i for i in range(len(x))])

The function $state\_preparation$ takes as input the numpy array which contains the binary representation of the input binary string. For example, for the binary string $010$, the numpy array would be [0 1 0].

The Pennylane function $BasisState$ takes as input, numpy array of the state to be initialized as the first parameter and the wires to which the state is to be assigned as the second parameter. This function can be written in a similar way as:

In [80]:
def state_preparation_2(x):
    for _, q in enumerate(state):
        if q == 1:
            qml.PauliX(wires=_)
        qml.Identity(wires=_)

From this new function $state\_preparation\_2$, we can see that the input numpy array is assigned in the form of MSB of the array to wire_0 and then progressively.

### 3. Variational Block

After we have initialized the input, we apply an operation to the prepared state which is analogous to the operation of assigning weights in classical machine learning.

In [81]:
def layer(layer_weights):
    for wire in range(len(layer_weights)):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([[i%(len(layer_weights)+1), (i+1)%(len(layer_weights)+1)] for i in range(len(layer_weights)+1)]):
        qml.CNOT(wires)

This operation puts each qubit in an entanglement with its neighboring qubits. This is done by the cyclic application of the $CNOT$ gates. We apply the $Rot$ function to apply a rotation in all the $X, Y, Z$ bases. This is to make sure that the qubit can be generalized and is free to attain any possible state. This make the model very general in its process to find the mapping and is not biased to a particular basis system.

### 4. Combining the Quantum part of the model

Having written the functions for state preparation and for the variational block, we can now combine both of them to form a final Quantum block.

In [82]:
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

The function returns the expectation values of each of the possible states for qubit at wire_0 in the $Z$ computational basis. This will prove useful as the $Z$ gate maps $0$ to $1$ and $1$ to $-1$. We can see that the difference is only in the sign of the states and we will use to get the output of the quantum block.

We also add a classical parameter $bias$ to the final quantum circuit which also a trainable parameter.

In [83]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

### 5. Cost function

Now that the main processing block of the model is done, we look at the training of the model. To do this, we define a classical cost function. This function will tell us how deviated is the result from the correct result. This is also called as the $Loss Function$. There are many types of loss functions. In this example, the standard square loss is used.

In [84]:
def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

The cost function is now defined as:

In [85]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

### 6. Optimization

Once the cost/loss of the predictions is calculated, it is passed through the optimizer to change the $weights$ of the veriational block by certain value. This value is decided by the optimizer. The optimizers try to find the optimum weights by find the local minima of an objective function. In this example, the $NesterovMomentumOptimizer$ is used with a momentum of $0.5$.

We first load the data for the training and evaluation of the model.

In [86]:
data = np.loadtxt("data/parity_train_2.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

# for x,y in zip(X, Y):
#     print(f"x = {x}, y = {y}")

We then define the necessary parameters for the weights of the layers, the number of layers, the number of qubits, etc. Initially, we can have the wights as randomly initialized as they will be optimized in the training process.

In [87]:
np.random.seed(0)
num_qubits = len(X[0])
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)

Weights: [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]
  [ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]
  [ 0.02269755 -0.01454366  0.00045759]
  [-0.00187184  0.01532779  0.01469359]]

 [[ 0.00154947  0.00378163 -0.00887786]
  [-0.01980796 -0.00347912  0.00156349]
  [ 0.01230291  0.0120238  -0.00387327]
  [-0.00302303 -0.01048553 -0.01420018]
  [-0.0170627   0.01950775 -0.00509652]
  [-0.00438074 -0.01252795  0.0077749 ]
  [-0.01613898 -0.0021274  -0.00895467]
  [ 0.00386902 -0.00510805 -0.01180632]
  [-0.00028182  0.00428332  0.00066517]
  [ 0.00302472 -0.00634322 -0.00362741]]]
Bias:  0.0


In [88]:
opt = qml.NesterovMomentumOptimizer(0.5)
batch_size = 30

We also set the batch_size parameter which trains the model on a batch of data in an iteration. This makes the training process faster as compared to training one row of data at a time.

We now define the accuracy of the model as the ratio of the number of inputs for which the correct label was predicted and the total number of inputs.

In [89]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [90]:
weights = weights_init
bias = bias_init
for it in range(50):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 2.0672117 | Accuracy: 0.5226190
Iter:    2 | Cost: 1.8848342 | Accuracy: 0.5226190
Iter:    3 | Cost: 1.7287990 | Accuracy: 0.5226190
Iter:    4 | Cost: 1.1225617 | Accuracy: 0.5226190
Iter:    5 | Cost: 1.0243291 | Accuracy: 0.5059524
Iter:    6 | Cost: 1.0133340 | Accuracy: 0.5059524
Iter:    7 | Cost: 1.0208218 | Accuracy: 0.4940476
Iter:    8 | Cost: 1.0001625 | Accuracy: 0.4773810
Iter:    9 | Cost: 1.0776565 | Accuracy: 0.4940476
Iter:   10 | Cost: 1.0028528 | Accuracy: 0.5059524
Iter:   11 | Cost: 1.0029288 | Accuracy: 0.5059524
Iter:   12 | Cost: 1.0147975 | Accuracy: 0.5059524
Iter:   13 | Cost: 1.0351790 | Accuracy: 0.5059524
Iter:   14 | Cost: 1.0061342 | Accuracy: 0.4940476
Iter:   15 | Cost: 1.0148540 | Accuracy: 0.5059524
Iter:   16 | Cost: 1.0649363 | Accuracy: 0.5059524
Iter:   17 | Cost: 1.0029948 | Accuracy: 0.5059524
Iter:   18 | Cost: 1.2067436 | Accuracy: 0.5059524
Iter:   19 | Cost: 1.0000805 | Accuracy: 0.4940476
Iter:   20 | Cost: 1.0000741 | 

Here, we do the training of the model. In each iteration, we divide the input data into batches for training. To make sure that in a given batch, the data is as random as possible, we use numpy's random function to create a batch of 5 rows. These rows are selected by the indices in the list generated by the np.random function.

We then optimize the weights and teh bias parameters to find the optimum solution in that iteration for the generated batch of data.

Using these parameters, predictions are made and then the cost is calculated. We also measure the accuracy of the model for the inputs and optimized parameters for that iteration.

These steps are repeated 100 times. However, this carries the risk of overfitting in the case where the number of iterations is so high to the point where the model has learned the mapping for the input data so well that it can only correctly label the input if the input is one of the training set values. Whereas, if any value which does not belong to the training set is passed, it performs poorly.

Therefore, we now have a look at the test set for the evaluation of the model:

In [91]:
data = np.loadtxt("data/parity_test_2.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

x = [0 0 0 0 0 0 0 1 0 0], y = 1, pred=-1.0
x = [0 0 0 0 0 0 0 1 1 1], y = 1, pred=-1.0
x = [0 0 0 0 0 1 1 1 1 0], y = -1, pred=-1.0
x = [0 0 0 0 1 0 0 0 1 0], y = -1, pred=-1.0
x = [0 0 0 0 1 0 0 0 1 1], y = 1, pred=-1.0
x = [0 0 0 0 1 0 1 0 1 1], y = -1, pred=-1.0
x = [0 0 0 0 1 0 1 1 1 1], y = 1, pred=-1.0
x = [0 0 0 0 1 1 0 0 0 1], y = 1, pred=-1.0
x = [0 0 0 0 1 1 0 0 1 0], y = 1, pred=-1.0
x = [0 0 0 0 1 1 0 0 1 1], y = -1, pred=-1.0
x = [0 0 0 0 1 1 0 1 0 0], y = 1, pred=-1.0
x = [0 0 0 0 1 1 1 0 0 0], y = 1, pred=-1.0
x = [0 0 0 1 0 0 0 1 0 0], y = -1, pred=-1.0
x = [0 0 0 1 0 0 1 0 0 1], y = 1, pred=-1.0
x = [0 0 0 1 0 0 1 0 1 0], y = 1, pred=-1.0
x = [0 0 0 1 0 0 1 0 1 1], y = -1, pred=-1.0
x = [0 0 0 1 0 0 1 1 1 0], y = -1, pred=-1.0
x = [0 0 0 1 0 1 0 0 0 0], y = -1, pred=-1.0
x = [0 0 0 1 0 1 0 1 0 1], y = -1, pred=-1.0
x = [0 0 0 1 0 1 1 1 1 1], y = -1, pred=-1.0
x = [0 0 0 1 1 0 0 1 1 0], y = -1, pred=-1.0
x = [0 0 0 1 1 1 0 0 1 0], y = -1, pred=-1.0
x = [0 0 0 1 1 1 0 0

In this case, the accuracy of prediction on unseen data is $0.47282608695652173$